In [1]:
import os
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import datetime

In [2]:
tweet_path = 'Raw_tweets'
store_path = 'Senti_score'

In [3]:
df = pd.read_csv('Ticker.csv')

In [4]:
ticker = list(df['Ticker'])

In [5]:
def sentimentScore(Tweet):
    analyzer = SentimentIntensityAnalyzer()
    results = []
    for sentence in Tweet:
        vs = analyzer.polarity_scores(sentence)
        results.append(vs)
    return results

In [6]:
for i in range(len(ticker)):
    
    name = ticker[i]+'_tweets.xlsx'
    file = os.path.join(tweet_path,name)
    tweet_df = pd.read_excel(file,index_col=0)
    Tweet = tweet_df['Text']
    df_results = pd.DataFrame(sentimentScore(Tweet))
    df_tweet = pd.merge(tweet_df['Time'],df_results,left_index=True, right_index=True)
    for j in range(len(df_tweet)):
        df_tweet.loc[j,'Time'] = df_tweet.loc[j,'Time'].date() 
    df_tweet['Score']=0
    df_tweet['Score'][df_tweet['compound']<0]=-1
    df_tweet['Score'][df_tweet['compound']>0]=1

    store_name = ticker[i] + '_senti_score.csv'
    save_path = os.path.join(store_path,store_name)
    df_tweet.to_csv(save_path)

<ipython-input-6-d5bb30455788>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweet['Score'][df_tweet['compound']<0]=-1
<ipython-input-6-d5bb30455788>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweet['Score'][df_tweet['compound']>0]=1
<ipython-input-6-d5bb30455788>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweet['Score'][df_tweet['compound']<0]=-1
<ipython-input-6-d5bb30455788>:13: SettingWithCopyWarning: 
A 

In [7]:
for i in range(len(ticker)):
    name = ticker[i]+'_senti_score.csv'
    file = os.path.join(store_path,name)
    senti_df = pd.read_csv(file,index_col=0)
    days=list(set(list(senti_df['Time'])))
    days.sort()
    P_N = pd.DataFrame()
    for j in range(len(days)):
        num_tw = senti_df[senti_df['Time'] == days[j]]['Score'].count()
        P_plus_N = abs(senti_df[senti_df['Time'] == days[j]]['Score']).sum()
        P_minue_N = senti_df[senti_df['Time'] == days[j]]['Score'].sum()
        P_N.loc['POS',days[j]] = (P_plus_N + P_minue_N)/2
        P_N.loc['NEG',days[j]] = (P_plus_N - P_minue_N)/2
        P_N.loc['num_tw',days[j]] = num_tw
        P_N.loc['SA',days[j]] = P_minue_N
        if P_plus_N == 0:
            P_N.loc['SR',days[j]] = 0
        else:
            P_N.loc['SR',days[j]] = P_minue_N/P_plus_N
    store_name = ticker[i] + '_sa_sr.csv'
    store_dir = 'SA_SR'
    save_path = os.path.join(store_dir,store_name)
    P_N.to_csv(save_path)

Calculate SA and SR of each stock every single day.

In [8]:
import yfinance as yf
import math

In [9]:
# Use SPY return as the market index
market_index=yf.download('SPY',start='2020-12-04',end='2020-12-15')

[*********************100%***********************]  1 of 1 completed


In [10]:
tra_day=list(market_index.index)
for j in range(len(tra_day)):
    if j > 0:
        market_index.loc[tra_day[j],'log_return']=math.log(market_index.loc[tra_day[j],'Adj Close']/market_index.loc[tra_day[j-1],'Adj Close'])

In [11]:
for i in range(len(ticker)):
    df_stock=yf.download(ticker[i],start='2020-12-04',end='2020-12-15')
    tra_day=list(df_stock.index)
    for j in range(len(tra_day)):
        high = df_stock.loc[tra_day[j],'High']
        low = df_stock.loc[tra_day[j],'Low']
        df_stock.loc[tra_day[j],'Vol'] = 2*(high-low)/(high+low)
        if j > 0:
            df_stock.loc[tra_day[j],'log_return']=math.log(df_stock.loc[tra_day[j],'Adj Close']/df_stock.loc[tra_day[j-1],'Adj Close'])
            df_stock.loc[tra_day[j],'Excess_return'] = df_stock.loc[tra_day[j],'log_return'] - market_index.loc[tra_day[j],'log_return']
    store_name = ticker[i] + '_price.csv'
    save_path = os.path.join('Stock_return_vol',store_name)
    df_stock.to_csv(save_path)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
